# Initialization
For data in CNTC lab only

In [1]:
from safedigital import temperature as TR  # 从safedigital包中导入temperature库
import pandas as pd
import os
os.chdir('../')
cur_dir = os.getcwd()
print(cur_dir)

def data_clean(test_date,device_id):
    sample_time = 1  # 降采样系数列表
    sample_str = '10s'  # 采样时间列表
    test_folder = test_date + device_id  # 试验文件夹
    folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
    folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
    file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
    file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
    path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
    path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
    raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
    raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
    syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳
    chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, 1)  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, 1)  # 热电偶1数据降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

    col_sen = list(syn_sen.columns)
    col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
    col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)


C:\Users\cnbofan1\ABB\Safe Digital in CN - Documents - Documents\06_Test and Analytics


# 20211019

In [ ]:
test_date = '20211019'  # 试验日期
device_id = '_TR SR40.5kV cable skin vert 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211029

In [ ]:
test_date = '20211029'  # 试验日期
device_id = '_DTR&TR_SR12kV 100A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211102

In [ ]:
test_date = '20211102'  # 试验日期
device_id = '_DTR&TR_3in1_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211103

In [ ]:
test_date = '20211103'  # 试验日期
device_id = '_DTR&TR_3in1_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211104

In [ ]:
test_date = '20211104'  # 试验日期
device_id = '_DTR&TR_3in1_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211105

In [ ]:
test_date = '20211105'  # 试验日期
device_id = '_DTR&TR_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211106

In [ ]:
test_date = '20211106'  # 试验日期
device_id = '_DTR&TR_SR12kV Current Variant'
sample_time_list = [90]  # 降采样系数列表
sample_str_list = ['15min']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211107

In [ ]:
test_date = '20211107'  # 试验日期
device_id = '_DTR&TR_SR12kV Current Variant'
sample_time_list = [90]  # 降采样系数列表
sample_str_list = ['15min']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211118


In [ ]:
test_date = '20211118'  # 试验日期
device_id = '_DTR&TR_SR12kV Current Drop'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211123


In [ ]:
test_date = '20211123'  # 试验日期
device_id = '_DTR&TR_SR12kV Fault Simulation'
sample_time_list = [30]  # 降采样系数列表
sample_str_list = ['5m']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
               'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211124


In [ ]:
test_date = '20211124'  # 试验日期
device_id = '_DTR&TR_SR12kV Fault Simulation'
sample_time_list = [30]  # 降采样系数列表
sample_str_list = ['5min']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
               'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211208


In [ ]:
test_date = '20211208'  # 试验日期
device_id = '_DTR&TR_SA12kV 300A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211209
Constant current 630A till steady-state


In [ ]:
test_date = '20211209'  # 试验日期
device_id = '_DTR&TR_SA12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211210
Constant current 100A till steady-state


In [ ]:
test_date = '20211210'  # 试验日期
device_id = '_DTR&TR_SA12kV 100A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211216
Variant current 100-300-500-630-400-200


In [ ]:
test_date = '20211216'  # 试验日期
device_id = '_DTR&TR_SA12kV Current Variant'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211217
Variant current 100-300-500-630-400-200


In [ ]:
test_date = '20211217'  # 试验日期
device_id = '_DTR&TR_SA12kV Current Variant'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211218
Constant current 500A till steady-state


In [ ]:
test_date = '20211218'  # 试验日期
device_id = '_DTR&TR_SA12kV 500A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211219
Variant current 100-300-500-630-400-200


In [ ]:
test_date = '20211219'  # 试验日期
device_id = '_DTR&TR_SA12kV Current Variant'
sample_time_list = [90]  # 降采样系数列表
sample_str_list = ['15min']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['date_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211220
Variant current with fault 100-300-500-630-400-200


In [ ]:
test_date = '20211220'  # 试验日期
device_id = '_DTR&TR_SA12kV Fault Simulation'
sample_time_list = [30]  # 降采样系数列表
sample_str_list = ['5min']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211221
630A constant current with fault 


In [ ]:
test_date = '20211221'  # 试验日期
device_id = '_DTR&TR_SA12kV Fault Simulation'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211222
630A-100A current drop without fault 


In [ ]:
test_date = '20211222'  # 试验日期
device_id = '_DTR&TR_SA12kV Fault Simulation'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

# 20220222

In [2]:
test_date = '20220222'  # 试验日期
device_id = '_TR_3in1_SR12kV Cable Skin'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

KeyError: 'datetime'

#20220224

In [3]:
test_date = '20220224'  # 试验日期
device_id = '_TR_3in1_SR12kV Cable Skin'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:45:42
logger ended at 18:30:32
RangeIndex(start=0, stop=3507, step=1)
sensor & couplers common start time =  2022-02-24 08:45:42
sensor & couplers common end time = 2022-02-24 18:28:30
number of duplicated index is :  0
0th group of data has no discontinued points
number of duplicated index is :  0
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3497


#20220225

In [4]:
test_date = '20220225'  # 试验日期
device_id = '_TR_3in1_SR12kV Cable Skin'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:53:08
test data are not fully recorded
RangeIndex(start=0, stop=2314, step=1)
sensor & couplers common start time =  2022-02-25 08:53:08
sensor & couplers common end time = 2022-02-25 15:13:08
number of duplicated index is :  0
0th group of data has no discontinued points
number of duplicated index is :  0
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 2280


#20220226

In [3]:
test_date = '20220226'  # 试验日期
device_id = '_TR_3in1_SR12kV Fault'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 09:52:24
test data are not fully recorded
RangeIndex(start=0, stop=2262, step=1)
sensor & couplers common start time =  2022-02-26 09:52:24
sensor & couplers common end time = 2022-02-26 16:06:30
number of duplicated index is :  0
0th group of data has no discontinued points
number of duplicated index is :  0
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 2245


#20220228

In [3]:
test_date = '20220228'  # 试验日期
device_id = '_TR_3in1_SR12kV Fault'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:56:53
logger ended at 19:08:52
RangeIndex(start=0, stop=3637, step=1)
sensor & couplers common start time =  2022-02-28 08:56:53
sensor & couplers common end time = 2022-02-28 19:01:50
number of duplicated index is :  0
0th group of data has no discontinued points
number of duplicated index is :  0
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3630


#20220302

In [4]:
test_date = '20220302'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:55:23
logger ended at 18:03:33
RangeIndex(start=0, stop=3278, step=1)
sensor & couplers common start time =  2022-03-02 08:55:23
sensor & couplers common end time = 2022-03-02 18:01:20
number of duplicated index is :  0
0th group of data has no discontinued points
number of duplicated index is :  0
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3276


#20220303

In [5]:
test_date = '20220303'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:59:06
logger ended at 17:21:26
RangeIndex(start=0, stop=3007, step=1)
sensor & couplers common start time =  2022-03-03 08:59:06
sensor & couplers common end time = 2022-03-03 17:18:50
number of duplicated index is :  0
0th group of data has no discontinued points
number of duplicated index is :  0
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 2999


#20220304

In [7]:
test_date = '20220304'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 09:14:28
logger ended at 17:48:31
RangeIndex(start=0, stop=3083, step=1)
sensor & couplers common start time =  2022-03-04 09:14:28
sensor & couplers common end time = 2022-03-04 17:46:20
number of duplicated index is :  14
0th group of data has discontinued points
number of duplicated index is :  14
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3072


#20220308

In [8]:
test_date = '20220308'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:52:32
logger ended at 17:51:44
RangeIndex(start=0, stop=3230, step=1)
sensor & couplers common start time =  2022-03-08 08:52:32
sensor & couplers common end time = 2022-03-08 17:49:00
number of duplicated index is :  4
0th group of data has discontinued points
number of duplicated index is :  4
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3219


#20220309

In [10]:
test_date = '20220309'  # 试验日期
device_id = '_TR_3in1_SR12kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:40:42
logger ended at 17:24:35
RangeIndex(start=0, stop=3151, step=1)
sensor & couplers common start time =  2022-03-09 08:40:42
sensor & couplers common end time = 2022-03-09 17:22:00
number of duplicated index is :  15
0th group of data has discontinued points
number of duplicated index is :  15
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3128


#20220316

In [11]:
test_date = '20220316'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 11:33:30
logger ended at 20:03:42
RangeIndex(start=0, stop=3063, step=1)
sensor & couplers common start time =  2022-03-16 11:33:30
sensor & couplers common end time = 2022-03-16 20:01:10
number of duplicated index is :  6
0th group of data has discontinued points
number of duplicated index is :  6
1th group of data has no discontinued points
sensor & couplers common start index : 0
sensor & couplers common end index : 3046


#20220317

In [18]:
test_date = '20220317'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:58:47
logger ended at 17:16:29
RangeIndex(start=0, stop=2985, step=1)
sensor & couplers common start time =  2022-03-17 08:58:47
sensor & couplers common end time = 2022-03-17 17:13:30
number of duplicated index is :  17
0th group of data has discontinued points
number of duplicated index is :  17
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 2969


#20220318

In [19]:
test_date = '20220318'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

logger started from 08:55:35
test data are not fully recorded
RangeIndex(start=0, stop=3088, step=1)
sensor & couplers common start time =  2022-03-18 08:55:35
sensor & couplers common end time = 2022-03-18 17:19:50
number of duplicated index is :  5
0th group of data has discontinued points
number of duplicated index is :  5
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3026


#20220322
100-300-500-630-400-200A

In [2]:
test_date = '20220322'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
data_clean(test_date,device_id)

logger started from 10:30:35
logger ended at 16:49:44
RangeIndex(start=0, stop=2280, step=1)
sensor & couplers common start time =  2022-03-22 10:31:10
sensor & couplers common end time = 2022-03-22 16:49:34
number of duplicated index is :  4
0th group of data has discontinued points
number of duplicated index is :  4
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 2271


#20220323
630-700-630A till stable

In [4]:
test_date = '20220323'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
data_clean(test_date,device_id)

logger started from 08:42:42
logger ended at 17:54:20
RangeIndex(start=0, stop=3306, step=1)
sensor & couplers common start time =  2022-03-23 08:42:50
sensor & couplers common end time = 2022-03-23 17:53:40
number of duplicated index is :  2
0th group of data has discontinued points
number of duplicated index is :  2
1th group of data has no discontinued points
sensor & couplers common start index : 1
sensor & couplers common end index : 3305
